*Note: You are currently reading this using Google Colaboratory which is a cloud-hosted version of Jupyter Notebook. This is a document containing both text cells for documentation and runnable code cells. If you are unfamiliar with Jupyter Notebook, watch this 3-minute introduction before starting this challenge: https://www.youtube.com/watch?v=inN8seMm7UI*

---

In this challenge, you need to create a machine learning model that will classify SMS messages as either "ham" or "spam". A "ham" message is a normal message sent by a friend. A "spam" message is an advertisement or a message sent by a company.

You should create a function called `predict_message` that takes a message string as an argument and returns a list. The first element in the list should be a number between zero and one that indicates the likeliness of "ham" (0) or "spam" (1). The second element in the list should be the word "ham" or "spam", depending on which is most likely.

For this challenge, you will use the [SMS Spam Collection dataset](http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/). The dataset has already been grouped into train data and test data.

The first two cells import the libraries and data. The final cell tests your model and function. Add your code in between these cells.


In [1]:

import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt



     ---------------------------------------- 4.2/4.2 MB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 48.8/48.8 KB 2.4 MB/s eta 0:00:00
     ---------------------------------------- 86.9/86.9 KB 2.5 MB/s eta 0:00:00
     -------------------------------------- 241.5/241.5 KB 1.5 MB/s eta 0:00:00
  Created wheel for promise: filename=promise-2.3-py3-none-any.whl size=21503 sha256=890f4edfe97d7c97f4b8191cb484cbdfcafde0d10810069c171bdcec47df842a
  Stored in directory: c:\users\nicus\appdata\local\pip\cache\wheels\54\4e\28\3ed0e1c8a752867445bab994d2340724928aa3ab059c57c8db
Successfully built promise


In [4]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

--2022-04-29 11:38:26--  https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.3.33, 172.67.70.149, 104.26.2.33
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.3.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 358233 (350K) [text/tab-separated-values]
Saving to: 'train-data.tsv'

     0K .......... .......... .......... .......... .......... 14%  265K 1s
    50K .......... .......... .......... .......... .......... 28% 38.1M 0s
   100K .......... .......... .......... .......... .......... 42% 33.4M 0s
   150K .......... .......... .......... .......... .......... 57%  369K 0s
   200K .......... .......... .......... .......... .......... 71% 20.6M 0s
   250K .......... .......... .......... .......... .......... 85% 6.11M 0s
   300K .......... .......... .......... .......... ......... 100% 6.69M=0.3s

2022-04-29 11:38:27 (1014 KB/s) - 'train-data.tsv' saved [358233/35

In [5]:
import csv
X_train = []
y_train = []
X_test = []
y_test = []
tsv_file = open(train_file_path)
read_tsv = csv.reader(tsv_file, delimiter="\t")

for txt in read_tsv:
  if txt[0] == "ham":
    y_train.append(0)
  else:
    y_train.append(1)
  X_train.append(txt[1])

tsv_file = open(test_file_path)
read_tsv = csv.reader(tsv_file, delimiter="\t")

for txt in read_tsv:
  if txt[0] == "ham":
    y_test.append(0)
  else:
    y_test.append(1)
  X_test.append(txt[1])

In [6]:
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=7700, split=' ')
tokenizer.fit_on_texts(X_train)
print(tokenizer.word_index)
X_tokens = tokenizer.texts_to_sequences(X_train)
X_pad = pad_sequences(X_tokens)
print(X_pad[0])

{'i': 1, 'to': 2, 'you': 3, 'a': 4, 'the': 5, 'u': 6, 'and': 7, 'in': 8, 'is': 9, 'me': 10, 'my': 11, 'for': 12, 'your': 13, 'it': 14, 'of': 15, 'call': 16, 'have': 17, 'on': 18, 'now': 19, 'that': 20, '2': 21, 'are': 22, 'so': 23, 'but': 24, 'not': 25, 'or': 26, 'do': 27, 'at': 28, 'can': 29, "i'm": 30, 'will': 31, 'be': 32, 'with': 33, 'if': 34, 'get': 35, 'just': 36, 'ur': 37, 'no': 38, 'this': 39, 'we': 40, '4': 41, 'gt': 42, 'lt': 43, 'up': 44, 'ok': 45, 'go': 46, 'free': 47, 'when': 48, 'how': 49, 'out': 50, 'all': 51, 'from': 52, 'what': 53, 'know': 54, 'like': 55, 'then': 56, 'good': 57, 'got': 58, 'come': 59, 'its': 60, 'time': 61, 'day': 62, 'only': 63, 'am': 64, 'was': 65, 'he': 66, 'love': 67, 'send': 68, 'there': 69, 'as': 70, 'text': 71, 'txt': 72, 'want': 73, 'going': 74, 'one': 75, 'by': 76, 'ã¼': 77, 'about': 78, 'need': 79, 'still': 80, 'lor': 81, 'n': 82, 'r': 83, "i'll": 84, 'home': 85, 'our': 86, 'stop': 87, 'she': 88, 'see': 89, 'today': 90, 'sorry': 91, 'reply': 

In [7]:
model = keras.Sequential()

model.add(tf.keras.layers.Embedding(7700,32))
model.add(tf.keras.layers.LSTM(32))
model.add(tf.keras.layers.Dense(1,'sigmoid'))

In [8]:
model.compile(loss = 'binary_crossentropy', optimizer='adam',metrics = ['accuracy'])

In [9]:
y_train = np.array(y_train)
y_test = np.array(y_test)

In [10]:
model.fit(X_pad,y_train,epochs=6,validation_split=0.2)

Epoch 1/6
105/105 [==============================] - 7s 52ms/step - loss: 0.2915 - accuracy: 0.9138 - val_loss: 0.1017 - val_accuracy: 0.9761
Epoch 2/6
105/105 [==============================] - 5s 47ms/step - loss: 0.0658 - accuracy: 0.9856 - val_loss: 0.0544 - val_accuracy: 0.9892
Epoch 3/6
105/105 [==============================] - 5s 48ms/step - loss: 0.0255 - accuracy: 0.9934 - val_loss: 0.0444 - val_accuracy: 0.9916
Epoch 4/6
105/105 [==============================] - 5s 48ms/step - loss: 0.0138 - accuracy: 0.9967 - val_loss: 0.0456 - val_accuracy: 0.9904
Epoch 5/6
105/105 [==============================] - 5s 49ms/step - loss: 0.0078 - accuracy: 0.9985 - val_loss: 0.0440 - val_accuracy: 0.9856
Epoch 6/6
105/105 [==============================] - 5s 48ms/step - loss: 0.0033 - accuracy: 0.9997 - val_loss: 0.0418 - val_accuracy: 0.9916


In [11]:

word_index = tokenizer.word_index

def encode(text):
  txt_tokens = keras.preprocessing.text.text_to_word_sequence(text)
  txt_tokens = [word_index[word] if word in word_index else 0 for word in txt_tokens]
  return(pad_sequences([txt_tokens],250)[0])

encoded = encode("you can never do nothing")
encoded

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   

In [12]:
reverse_word_index = {value : key for (key,value) in word_index.items()}
def decode(integers):
  PAD = 0
  text = ""
  for num in integers:
    if num != PAD:
      text += reverse_word_index[num]+" "
  return text[:-1]

print(decode(encoded))

you can never do nothing


In [13]:
def predict(text):
  encoded_text = encode(text)
  pred = np.zeros((1,250))
  pred[0] = encoded_text
  result = model.predict(pred)
  return(result)

predict("I loved this movie")

array([[0.00114399]], dtype=float32)

In [14]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(text):
  encoded_text = encode(text)
  pred = np.zeros((1,250))
  pred[0] = encoded_text
  prediction = model.predict(pred)[0]
  label = ""
  if prediction <= 0.1:
    label = "ham"
  else:
    label = "spam"
  
  result = []
  result.append(prediction[0])
  result.append(label)
  return result

test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"]
                
for msg in test_messages:
  prediction = predict_message(msg)
  print(prediction)  

[0.00077956915, 'ham']
[0.51634675, 'spam']
[0.00044500828, 'ham']
[0.99329317, 'spam']
[0.9918313, 'spam']
[0.00040617585, 'ham']
[0.0005197823, 'ham']


In [15]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()


You passed the challenge. Great job!
